In [ ]:
from gcp_client import *
from read_gdoc_helper_functions import *
from write_table_helper_functions import *

In [ ]:
GCP = GCPAuth("./gcp_secrets")

#read client token
#refresh token if expired
#if token not found, prompt user to generate token
creds = GCP.creds
oauth = GCP.oauth

gdoc_client = GDocsClient(creds)
gsheet_client = GSheetsClient(oauth)
gdrive_client = GDriveClient(creds)

**gdoc methods**

blank template google doc with title placeholder and table_id string

![Alt text](screenshots/img1.png)

In [3]:
#read google doc
doc_id = ""
doc = gdoc_client.read_doc(doc_id)

Serving document: My Document


In [ ]:
#table_id as unique identifier. if same id is used then first table instance returned
table_id = "<<table_id>>"

#row index of header row (1 if single title row above headers)
header_index = 1

#get table body
table_body = read_table_section(doc, table_id)

In [5]:
df = pd.DataFrame({"animal": ["mouse", "dog", "hippo"],
                  "size": ["small", "medium", "big"],
                  })

In [ ]:
#insert empty rows
insert_row_request = insert_rows(table_body, df, header_index)
reply = gdoc_client.execute_request(doc_id, insert_row_request)

#reread doc with inserted empty rows to get cell indexes
doc = gdoc_client.read_doc(doc_id)
table_body = gdoc_client.read_table_section(doc, table_id)

#insert cell text
insert_text_request = insert_text(table_body, df, header_index)

#replace title placeholder
replace_title_request = replace_title_placeholder("<<title>>", "Table of animals and size")

batch_request = insert_text_request + replace_title_request
reply = gdoc_client.execute_request(doc_id, batch_request)

Serving document: My Document


populated template

![Alt text](screenshots/img2.png)

In [ ]:
#read doc table
doc = gdoc_client.read_doc(doc_id)
df = read_doc_table(doc, table_id, header_index)

#clean formatting objects. gdoc appends "\n" to all cell textruns
df = clean_table(df)
df

Serving document: My Document


,animal,size
0,mouse,small
1,dog,medium
2,hippo,big


**gsheet methods**

In [ ]:
sheet_id = ""
sheet_name = ""

#create new sheet tab in existing gsheet file and return worksheet
_, ws = gsheet_client.read_sheet(sheet_id, sheet_name)

#write dataframe to new sheet tab
gsheet_client.write_sheet(df, ws)

In [4]:
#read dataframe from sheet
df, _ = gsheet_client.read_sheet(sheet_id, sheet_name)
df

Serving sheet: My sheet, Sheet1


,animal,size
0,mouse,small
1,dog,medium
2,hippo,large


**gdrive methods**

In [ ]:
#get contents of a gdrive directory
directory_id = ""
parent_directory_id = ""

content = gdrive_client.gdrive_ls(directory_id, parent_directory_id)

In [ ]:
#create object in gdrive
filename = "a google doc"
mime_type = "gdoc"
parent_directory_id = ""

file_id = gdrive_client.create_gdrive_object(filename, mime_type, parent_directory_id)

#create a copy
gdrive_client.copy_file(file_id)